In [490]:
import os, sys, re, json, time, unittest
import itertools, collections
from importlib import reload

import numpy as np
import pandas as pd
from scipy import stats

import nltk
from collections import defaultdict
import os
from nltk import word_tokenize
import time
import scipy.stats

In [35]:
os.chdir('../../datasets/1-billion-word-LM_corpus')
filenames = []
for filename in os.listdir(os.getcwd()):
    filenames.append(filename)
file1 = open(filename).read()
print(file1[:200].lower())

print(word_tokenize(file1[:200]))

marshall reported from seattle ; jimenez reported from san jose , costa rica ; associated press writers yuri kageyama and mari yamaguchi in tokyo , jeremiah marquez in hong kong , ray lilley in wellin
['Marshall', 'reported', 'from', 'Seattle', ';', 'Jimenez', 'reported', 'from', 'San', 'Jose', ',', 'Costa', 'Rica', ';', 'Associated', 'Press', 'writers', 'Yuri', 'Kageyama', 'and', 'Mari', 'Yamaguchi', 'in', 'Tokyo', ',', 'Jeremiah', 'Marquez', 'in', 'Hong', 'Kong', ',', 'Ray', 'Lilley', 'in', 'Wellin']


## Create context totals/counts

In [36]:
def train_ngram(corpus_as_string):
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    context_totals = dict()
    corpus_tokenized = word_tokenize(corpus_as_string.lower())
    
    w_1, w_2 = None, None
    wordset = set()
    for word in corpus_tokenized:
        wordset.add(word)
        if w_1 is not None and w_2 is not None:
            counts[(w_2,w_1)][word] += 1
            # Update context
        w_2 = w_1
        w_1 = word
    for key, value in counts.items():
        context_totals[key] = sum(value.values())
    return context_totals, counts

In [37]:
context_totals, counts = train_ngram(file1, 4)

In [44]:
print(context_totals[('is', 'the')])
print(counts[('is', 'the')].get('capital'))

3548.0
11.0


## Get synonyms from sample sentences

In [51]:
#os.chdir('/smartThesaurus')
os.chdir('Code')
!pwd
sent_1= pd.read_csv('./sentence_generator/output.txt', sep = ',', names = ['pair', 'word', 'sentence'])
sent_2= pd.read_csv('./sentence_generator/output_2.txt', sep = ',', names = ['pair', 'word', 'sentence'])

/Users/nathanielschub/Desktop/MIDS/W266/smartThesaurus/Code


In [86]:
word_index = sen.index(word)

In [126]:
words_1 = sent_1.word
words_2 = sent_2.word
sents_1= sent_1.sentence
sents_2 = sent_2.sentence

contexts_list_1 = []
contexts_list_2 = []

for i in range(len(words_1)):
    word= words_1[i]
    sen = sents_1[i].split()
    word_index = sen.index(word)
    if word_index > 1:
        context = (sen[word_index-2], sen[word_index-1])
    elif word_index == 1:
        context = ("_", sen[word_index-1])
    else:
        context = ("_", "_")
    contexts_list_1.append(context)
    
for i in range(len(words_1)):
    word= words_2[i]
    sen = sents_2[i].split()
    word_index = sen.index(word)
    if word_index > 1:
        context = (sen[word_index-2], sen[word_index-1])
    elif word_index == 1:
        context = (".", sen[word_index-1])
    else:
        context = ("_", ".")
    contexts_list_2.append(context)

In [125]:
def eval(target_word, context_tuple, k, corpus_as_string):
    V = len(np.unique(corpus_as_string.lower().split()[:100]))
    x = target_word
    context = context_tuple
    C_abc = 0.0
    if counts.get(context, 0.0) == 0.0:
        C_abc = 0.0
    else:
        C_abc = counts.get(context, 0.0).get(x, 0.0)
                
    C_ab = context_totals.get(context, 0.0)
        
    return (C_abc +k)/(C_ab +(k*V))

In [348]:
scores_1_1 = []
scores_1_2 = []
scores_2_1 = []
scores_2_2 = []
start = time.time()
for i in range(len(sents_1)):
    if i%10==0:
        scores_1_1.append((words_1[i],eval(words_1[i], contexts_list_1[i],1, file1)))
        scores_1_2.append((words_1[i],eval(words_2[i], contexts_list_2[i],1, file1)))
        scores_2_1.append((words_2[i],eval(words_2[i], contexts_list_1[i],1, file1)))
        scores_2_2.append((words_2[i],eval(words_2[i], contexts_list_2[i],1, file1)))

        if i%100==0:
            print(i,time.time()-start)


0 1.4390640258789062
100 15.640044927597046


KeyboardInterrupt: 

## Run similarites

In [389]:
word_means_1_1 = []
running_mean_1_1=[]
for i in range(len(scores_1_1)):
    if (i+1)%10!=0:        
        running_mean_1_1.append(scores_1_1[i][1])
    else:
        running_mean_1_1.append(scores_1_1[i][1])
        word_means_1_1.append((scores_1_1[i][0],np.mean(running_mean_1_1)))
        running_mean_1_1=[]
        
word_means_1_2 =[]
running_mean_1_2=[]
for i in range(len(scores_1_1)):
    if (i+1)%10!=0:        
        running_mean_1_2.append(scores_1_2[i][1])
    else:
        running_mean_1_2.append(scores_1_2[i][1])
        word_means_1_2.append((scores_1_2[i][0],np.mean(running_mean_1_2)))
        running_mean_1_2=[]
 
word_means_2_1 =[]
running_mean_2_1=[]
for i in range(len(scores_1_1)):
    if (i+1)%10!=0:        
        running_mean_2_1.append(scores_2_1[i][1])
    else:
        running_mean_2_1.append(scores_2_1[i][1])
        word_means_2_1.append((scores_2_1[i][0],np.mean(running_mean_2_1)))
        running_mean_2_1=[]
        
word_means_2_2 =[]
running_mean_2_2=[]
for i in range(len(scores_1_1)):
    if (i+1)%10!=0:
        running_mean_2_2.append(scores_2_2[i][1])
    else:
        running_mean_2_2.append(scores_2_2[i][1])
        word_means_2_2.append((scores_2_2[i][0],np.mean(running_mean_2_2)))
        running_mean_2_2=[]
        
        


In [499]:
comparison_a=[]
comparison_b=[]

for i in range(len(word_means_1_1)):
    comparison_a.append(((word_means_1_1[i][0], word_means_2_2[i][0]), abs(abs(word_means_1_1[i][1])-abs(word_means_2_1[i][1]))))
    comparison_b.append(((word_means_1_1[i][0], word_means_2_2[i][0]), abs(abs(word_means_2_2[i][1])-abs(word_means_1_2[i][1]))))

distances={}
dist =[]
for i in range(len(comparison_a)):
#larger must be better for distances
    distance= (comparison_a[i][1]+comparison_b[i][1])/2

    distances[comparison_a[i][0]] = 1- distance
    dist.append(1-distance)
for i, wpair in zip(range(0,4),distances):
    print("{:30s} : {:6.4f}".format(str(wpair), distances[wpair]))


('childish', 'foolish')        : 1.0000
('child', 'boy')               : 0.9990
('child', 'adult')             : 0.9995
('author', 'creator')          : 0.9988


## Eval with Simlex

In [500]:
# load evaluation datsets
import os
#os.chdir('../datasets/SimLex-999')

path_evalsetSlex = 'SimLex-999.txt'
# path_evalsetWsim = '..\\datasets\\WordSim-353\\combined.csv'
# path_evalsetMens = '..\\datasets\\MEN\\MEN_dataset_natural_form_full'

evalsetSlex = pd.read_csv(path_evalsetSlex, sep='\t')
# evalsetWsim = pd.read_csv(path_evalsetWsim, sep=',')
# evalsetMens = pd.read_csv(path_evalsetMens, sep=' ', header=None)
# evalsetMens.columns = ['word1', 'word2', 'MEN_score']

print("\nSimLex eval set:")
display(evalsetSlex.head())
# print("\nWordSim eval set:")
# display(evalsetWsim.head())
# print("\nMEN eval set:")
# display(evalsetMens.head())

evalset_name_list = ['SimLex-999'] #, 'WordSim-353', 'MEN (3000)']


SimLex eval set:


,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93


In [501]:
# ranking the scores

Slex_sorted = evalsetSlex.copy()
print("\nSimLex-999 eval dataset sample:")
display(Slex_sorted.head())
Slex_sorted = Slex_sorted.filter(items=['word1', 'word2', 'SimLex999', 'SD(SimLex)'], axis=1)
Slex_sorted = Slex_sorted.sort_values(by=['SimLex999'], ascending=False)
Slex_sorted = Slex_sorted.set_index(pd.Index(range(0,Slex_sorted.shape[0])))

# Wsim_sorted = evalsetWsim.copy()
# print("\nWordSim-353 eval dataset sample:")
# display(Wsim_sorted.head())
# Wsim_sorted = Wsim_sorted.sort_index(by=['Human (mean)'], ascending=False)
# Wsim_sorted = Wsim_sorted.set_index(pd.Index(range(0,Wsim_sorted.shape[0])))
# 
# Mens_sorted = evalsetMens.copy()
# print("\nMEN (3000) eval dataset sample:")
# display(Mens_sorted.head())
# Mens_sorted = Mens_sorted.sort_index(by=['MEN_score'], ascending=False)
# Mens_sorted = Mens_sorted.set_index(pd.Index(range(0,Mens_sorted.shape[0])))

# add the rank column
print('*'*80)
rank = 0
Slex_sorted['rank_simlex'] = Slex_sorted.index + 1
print("\nTop and bottom rows of ranked SimLex eval dataset:")
display(pd.concat([Slex_sorted.head(), Slex_sorted.tail()], axis=0))

# rank = 0
# Wsim_sorted['rank_wordsim'] = Wsim_sorted.index + 1
# print("\nTop and bottom rows of ranked WordSim eval dataset:")
# display(pd.concat([Wsim_sorted.head(), Wsim_sorted.tail()], axis=0))
# 
# rank = 0
# Mens_sorted['rank_men'] = Mens_sorted.index + 1
# print("\nTop and bottom rows of ranked MEN eval dataset:")
# display(pd.concat([Mens_sorted.head(), Mens_sorted.tail()], axis=0))



SimLex-999 eval dataset sample:


,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93


********************************************************************************

Top and bottom rows of ranked SimLex eval dataset:


,word1,word2,SimLex999,SD(SimLex),rank_simlex
0,vanish,disappear,9.80,0.46,1
1,quick,rapid,9.70,1.14,2
2,creator,maker,9.62,1.40,3
3,stupid,dumb,9.58,1.48,4
4,insane,crazy,9.57,0.92,5
994,gun,fur,0.30,1.80,995
995,chapter,tail,0.30,1.57,996
996,dirty,narrow,0.30,0.89,997
997,new,ancient,0.23,0.46,998
998,shrink,grow,0.23,1.20,999


In [502]:
# Evaluating model's scoring on eval dataset(s)

# Simlex
ngram_Slex = Slex_sorted.copy()
ngram_Slex['ngram_score'] = Slex_sorted.apply(lambda row: distances[(row['word1'], row['word2'])], axis=1 )
ngram_Slex_sorted = ngram_Slex.sort_values(by=['ngram_score'], ascending=False)
ngram_Slex_sorted = ngram_Slex_sorted.set_index(pd.Index(range(0,ngram_Slex_sorted.shape[0])))
ngram_Slex_sorted['rank_ngram'] = ngram_Slex_sorted.index + 1
ngram_Slex_sorted = ngram_Slex_sorted.sort_values(by=['SimLex999'], ascending=False)
ngram_Slex['rank_ngram'] = ngram_Slex_sorted['rank_ngram'].values
display(ngram_Slex[:10])

# # Wordsim
# elmo_Wsim = Wsim_sorted.copy()
# elmo_Wsim['elmo_score'] = Wsim_sorted.apply(lambda row: elmo.similarity(row['Word 1'], row['Word 2']), axis=1 )
# elmo_argsort = (-elmo_Wsim['elmo_score']).argsort()   # to sort in descending order
# elmo_Wsim['rank_elmo'] = elmo_argsort + 1
# 
# # MEN
# elmo_Mens = Mens_sorted.copy()
# elmo_Mens['elmo_score'] = Mens_sorted.apply(lambda row: elmo.similarity(row['word1'], row['word2']), axis=1 )
# elmo_argsort = (-elmo_Mens['elmo_score']).argsort()   # to sort in descending order
# elmo_Mens['rank_elmo'] = elmo_argsort + 1

ngram_spearman = list()
ngram_spearman.append(scipy.stats.spearmanr(ngram_Slex['rank_simlex'], ngram_Slex['rank_ngram'])[0])
# elmo_spearman.append(scipy.stats.spearmanr(elmo_Wsim['rank_wordsim'], elmo_Wsim['rank_elmo'])[0])
# elmo_spearman.append(scipy.stats.spearmanr(elmo_Mens['rank_men'], elmo_Mens['rank_elmo'])[0])

print('*'*70)
ngram_spearman_df = pd.DataFrame({'Dataset': evalset_name_list, 'Spearman rank coeff.': ngram_spearman})
display(ngram_spearman_df)

,word1,word2,SimLex999,SD(SimLex),rank_simlex,ngram_score,rank_ngram
0,vanish,disappear,9.80,0.46,1,1.000000,1
1,quick,rapid,9.70,1.14,2,0.999613,280
2,creator,maker,9.62,1.40,3,0.999376,370
3,stupid,dumb,9.58,1.48,4,0.998607,557
4,insane,crazy,9.57,0.92,5,1.000000,26
5,large,big,9.55,0.44,6,0.998949,677
6,happy,cheerful,9.55,2.18,7,0.997829,477
7,cow,cattle,9.52,0.79,8,0.996512,791
8,area,region,9.47,0.58,9,0.995170,615
9,large,huge,9.47,1.27,10,0.998236,865


TypeError: 'list' object is not callable

In [503]:
scipy.stats.spearmanr(ngram_Slex['rank_simlex'], ngram_Slex['rank_ngram'])

SpearmanrResult(correlation=0.09598560484332026, pvalue=0.002389218824769293)

In [505]:
display(ngram_Slex[-10:])


,word1,word2,SimLex999,SD(SimLex),rank_simlex,ngram_score,rank_ngram
989,container,mouse,0.30,0.95,990,0.999988,574
990,ankle,window,0.30,1.80,991,0.998273,866
991,cliff,tail,0.30,1.57,992,0.998870,210
992,hole,agreement,0.30,1.72,993,0.995155,608
993,island,task,0.30,0.96,994,0.997518,403
994,gun,fur,0.30,1.80,995,0.999802,174
995,chapter,tail,0.30,1.57,996,0.999876,63
996,dirty,narrow,0.30,0.89,997,0.999251,500
997,new,ancient,0.23,0.46,998,0.996425,359
998,shrink,grow,0.23,1.20,999,0.999412,803


In [170]:
def eval_candidates(corpus_tokenized, list_target_words, context, k):
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    context_totals = dict()
    w_1, w_2 = None, None
    wordset = set()
    for word in corpus_tokenized:
        wordset.add(word)
        if w_1 is not None and w_2 is not None:
            counts[(w_2,w_1)][word] += 1
            # Update context
        w_2 = w_1
        w_1 = word
    for key, value in counts.items():
        context_totals[key] = sum(value.values())
    
    words = list(wordset)
    V = len(words)
    
    values=[]
    
    for word_x in list_target_words:
    
        C_abc = 0.0
        if counts.get(context, 0.0) == 0.0:
            C_abc = 0.0
        else:
            C_abc = counts.get(context, 0.0).get(word_x, 0.0)

        C_ab = context_totals.get(context, 0.0)

        values.append((word_x,(C_abc +k)/(C_ab +(k*V)) )) 
    return values


